**DRIVE MOUNT**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**IMPORTS**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from skimage.io import imread

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models

from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import pandas as pd
import os

from torchvision.utils import draw_segmentation_masks
from torch.functional import Tensor

import cv2

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

currentDirPath = "./drive/MyDrive/Software udvikling/DeepLearning/Eksamen/"

**MASKING** (virker ikke, men tror det er tæt på)

**ARTICLE MASKING FUNCTIONS** (Got it to work)

In [ ]:
#https://towardsdatascience.com/bounding-box-prediction-from-scratch-using-pytorch-a8525da51ddc
#https://jovian.ai/aakanksha-ns/road-signs-bounding-box-prediction
def create_mask(bounding_box_coord, image):
    """Creates a mask for the bounding box of same shape as image"""
    *_,rows,cols = image.shape
    mask = np.zeros((rows, cols))
    bb = bounding_box_coord.astype(np.int)
    mask[bb[0]:bb[2], bb[1]:bb[3]] = 1.
    return mask

def mask_to_bb(mask):
    """Convert mask Y to a bounding box, assumes 0 as background nonzero object"""
    cols, rows = np.nonzero(mask)
    if len(cols)==0: 
        return np.zeros(4, dtype=np.float32)
    top_row = np.min(rows)
    left_col = np.min(cols)
    bottom_row = np.max(rows)
    right_col = np.max(cols)
    return np.array([left_col, top_row, right_col, bottom_row], dtype=np.float32)


def create_bb_array(csvRow):
    """Generates bounding box array from a train_df row"""
    return np.array([csvRow['new_bbs_ymin'],csvRow['new_bbs_xmin'],csvRow['new_bbs_ymax'],csvRow['new_bbs_xmax']])


**VARIABLES FOR TEST**

In [ ]:
#CSV PATH
csvPathTest = currentDirPath + "TrainingData/D1/D1.csv"
#Read CSV
csvTest = pd.read_csv(csvPathTest)
#PULL FIRST ROW
firstImageTest = csvTest.iloc[0]
#CONSTRUCT IMAGE PATH
imgPathTest = currentDirPath + "TrainingData/D1/Images/" + firstImageTest['img_id']
#READ IMAGE
imageTest = read_image(imgPathTest)

**TESTING MAKS FUNCTIONS**

In [ ]:
#RETRIEVE BB COORDS
bbTest = create_bb_array(firstImageTest)
#CREATE MASK
maskTest = create_mask(bbTest, imageTest)
#PRINT BB IMAGE IN PLOT
plt.imshow(maskTest, cmap='gray')

**Save Mask**

In [ ]:
#df_targets = pd.read_csv(currentDirPath + "TrainingData/D1/D1.csv")
df_targets = pd.read_csv(currentDirPath + "TrainingData/D1/D1NewBBValue.csv")
for index, row in df_targets.iterrows():
  #CONSTRUCT IMAGE PATH
  imgPath = currentDirPath + "TrainingData/D1/Images/" + row['img_id']
  #READ IMAGE
  image = read_image(imgPath)
  #RETRIEVE BB COORDS
  bb = create_bb_array(row)
  #CREATE MASK
  mask = create_mask(bb, image)
  mask = torch.tensor(mask, dtype=torch.float32)
  #print(mask)
  #SAVE MASK
  save_image(mask, currentDirPath + "TrainingData/D1/Target_Images/" + row['img_id'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


KeyboardInterrupt: ignored

**RESIZE AND SAVE NEW BOUNDING BOX IMAGE FUNCTION**

In [ ]:
def resize_image_bb(read_path,write_path,row,size):
    """Resize an image and its bounding box and write image to new path"""
    bounding_box = create_bb_array(row)
    img = read_image(read_path)
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size),
        transforms.ToTensor()
    ])
    im_resized = transform(img)
    #print(im_resized.shape)
    mask = create_mask(bounding_box, img)
    #print(mask.shape)
    Y_resized = cv2.resize(mask, size)
    #print(Y_resized)
    new_path = currentDirPath + write_path + row['img_id']
    #print(new_path)
    save_image(im_resized, new_path)
    return write_path + row['img_id'], mask_to_bb(Y_resized)

**TEST REZISE METHOD**

In [ ]:
new_path,new_bb = resize_image_bb(imgPathTest, "TrainingData/D1/Images_Resized_bb/", firstImageTest,(200,300))
print(new_path, new_bb)    

**GET DATA TO MODIFY**

In [ ]:
csv = currentDirPath + "TrainingData/D1/D1.csv"
imgDir = currentDirPath + "TrainingData/D1/Images/"

df_train = pd.read_csv(csv)

**LOOP THROUGH ALL IMAGES AND MAKE RESIZED BOUNDING BOX** (RUN ONLY IF YOU NEED TO CHANGE THE SIZES)

In [ ]:
new_paths = []
new_bbs_ymin = []
new_bbs_xmin = []
new_bbs_ymax = []
new_bbs_xmax = []
train_path_resized = "TrainingData/D1/Images_Resized_bb/"
count = 0
for index, row in df_train.iterrows():
    #print(row.values)
    #print(row['img_id'])
    read_path = imgDir + row['img_id']
    new_path,new_bb = resize_image_bb(read_path, train_path_resized, row,(200,300))
    new_paths.append(new_path)
    #new_bbs.append(new_bb)
    new_bbs_ymin.append(new_bb[0])
    new_bbs_xmin.append(new_bb[1])
    new_bbs_ymax.append(new_bb[2])
    new_bbs_xmax.append(new_bb[3])
df_train['new_path'] = new_paths
#df_train['new_bb'] = new_bbs
df_train['new_bbs_ymin'] = new_bbs_ymin
df_train['new_bbs_xmin'] = new_bbs_xmin
df_train['new_bbs_ymax'] = new_bbs_ymax
df_train['new_bbs_xmax'] = new_bbs_xmax

NameError: ignored

**SAVE NEW BOUNDING BOX VALUES** (RUN ONLY IF YOU'VE RUN THE PREVIOUS LOOP)

In [ ]:
csv_path_new_values = currentDirPath + "TrainingData/D1/D1NewBBValue.csv"
df_train.to_csv(csv_path_new_values)

**LOAD CSV WITH BB VALUES**

In [ ]:
csv_path_new_values = currentDirPath + "TrainingData/D1/D1NewBBValue.csv"
df_train = pd.read_csv(csv_path_new_values)

**CHECK NEW VALUES IN DF_TRAIN**

In [ ]:
print(df_train[['new_path', 'new_bbs_ymin']])

**SPLIT SET INTO TRAINING AND VALIDATION**

In [ ]:
df_train = df_train
x_train, x_val = train_test_split(df_train[['img_id', 'ymin','xmin','ymax','xmax']], test_size=0.2, random_state=18)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)

# **DATASET**

In [ ]:
class CarsDataset(Dataset):
    def __init__(self, data, ActivateTransforms=False, width=200, height=300):
        self.transforms = ActivateTransforms
        self.data = data
        self.resizeWidth = width
        self.resizeHeight = height
        self.transformCompose = transforms.Compose([
              transforms.ToPILImage(),
              torchvision.transforms.Resize((self.resizeHeight,self.resizeWidth)),
              transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.data['img_id'])
    
    def __getitem__(self, idx):
        #print(idx)
        imagePath = currentDirPath + "TrainingData/D1/Images/"+ self.data['img_id'][idx]
        #print(imagePath)
        image = read_image(imagePath)
        if(self.transforms):
          image = self.transformCompose(image)
        #print(self.data['new_bbs_ymin'][idx])
        bb = torch.Tensor([self.data['ymin'][idx]/self.resizeHeight,self.data['xmin'][idx]/self.resizeWidth,self.data['ymax'][idx]/self.resizeHeight,self.data['xmax'][idx]/self.resizeWidth])
        
        #bb = self.transformCompose2(bb)
        #print(bb)
        #print(type(image))
        #bb = torch.tensor(bb, dtype=torch.float32)
        #bb = bb.unsqueeze(-1)
        
        return image, bb 

# **DATALOADER**

In [ ]:
carsTrainSet = CarsDataset(x_train, ActivateTransforms=True)
carsValSet = CarsDataset(x_val, ActivateTransforms=False)

carsTrainLoader = torch.utils.data.DataLoader(carsTrainSet, batch_size=4,
                                        shuffle=True, num_workers=2)
carsValLoader = torch.utils.data.DataLoader(carsValSet, batch_size=4,
                                         shuffle=True, num_workers=2)


#**MODEL**

Resnet trænet til at tage input som billede i mini batches af 3 kanaler rgb (3 x H x W)
https://pytorch.org/hub/pytorch_vision_resnet/

In [ ]:
class CarsModel(nn.Module):
    def __init__(self):
        super(CarsModel, self).__init__()        
        #resnet = models.resnet34(pretrained=True)
        #layers = list(resnet.children())[:8]
        # self.features1 = nn.Sequential(*layers[:6])
        # self.features2 = nn.Sequential(*layers[6:])
        # self.classifier = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 4))
        #self.bb = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 4))
        # self.featuresStart = nn.Linear(300, 1000)
        # self.features1 = nn.Linear(1000, 512)
        # self.features2 = nn.Linear(512, 256)
        # self.features3 = nn.Linear(256, 128)
        # self.features4 = nn.Linear(128, 64)
        # self.features5 = nn.Linear(64, 32)
        # self.features6 = nn.Linear(32, 16)
        # self.features7 = nn.Linear(16, 8)
        # self.features8 = nn.Linear(8,4)
        
        self.net = torch.nn.Sequential(
            torch.nn.Flatten(start_dim=0),
            torch.nn.Linear((200 * 300) * 3, 1000),
            torch.nn.Linear(1000, 500),
            torch.nn.ReLU(),
            torch.nn.Linear(500, 250),
            torch.nn.ReLU(),
            torch.nn.Linear(250, 4),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        # x = self.featuresStart(x)
        # x = self.features1(x)
        # x = self.features2(x)
        # x = self.features3(x)
        # x = self.features4(x)
        # x = self.features5(x)
        # x = self.features6(x)
        # x = self.features7(x)
        # x = self.features8(x)
        # x = F.relu(x)
        #x = nn.AdaptiveAvgPool2d((1,1))(x)
        #x = x.view(x.shape[0], -1)
        x = self.net(x)
        return x #self.classifier(x),self.bb(x)

In [ ]:
net = CarsModel()
#Loss function 
criterion = nn.MSELoss()#nn.CrossEntropyLoss()#F.binary_cross_entropy#nn.KLDivLoss()
#Optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# **TRAINING** 

Running as well as the other network, no clue why, no clue how, fuck this garbage.
Key error happens every now and again, no fucking clue either. prob the shitty split made earlier idk
https://towardsdatascience.com/creating-and-training-a-u-net-model-with-pytorch-for-2d-3d-semantic-segmentation-model-building-6ab09d6a0862

In [ ]:
running_loss = 0.0
for epoch in range(100):  # loop over the dataset multiple times

    for i, data in enumerate(carsTrainLoader, 0):

        inputs, labels = data
        #for batch_index in range(len(inputs)):
                #input, target = inputs[batch_index], labels[batch_index]
                #input = input.unsqueeze(0)
                #target = target.unsqueeze(0)
        #print(labels)
        #print(labels.shape)
        #print(inputs)
        #print(inputs.shape)
        optimizer.zero_grad()
        outputs = net(inputs)
        #print(outputs.shape)
        #print(outputs)
        #print(labels)
        #print(classifier)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        print("Loss: " + str(running_loss))
    print("Current Epoch:" + epoch)

print('Finished Training')

RuntimeError: ignored